Difference  between Boosting and Bagging technique -                                                             
Bagging involes training multiple enstenses of the same based learning algorithm on different subset of training data where ech subset is created by sampling the training data with replacement after training the prediction ofrom each model are typically averge for regression or majority voted for classification to produced the final prediction the main goal of bagging is to reduce varieance by averging over multiple noise model and improving overall model stabilty and generlisation.

Boosting involves sequentially training a series of weak learners often decision trees where each learner focuses on the mistakes made by its predecessors.                                                             
In each iteration the algorithm assigns higher weights to the instances that were misclassified by the previous models.         
The final model is a weighted combination of all weak learners where each learner contributes differently based on its performance.                                                              
Boosting aims to reduce both bias and variance by iteratively focusing on the most difficult to predict instances resulting in a strong classifier that performs well on the entire dataset.                                                     


Types of Boosting -                                           
1. Adaptive Boosting -                                                         
In adaptive boosting, only decision tree stumps are created.                                            
                                                                    
                                                                    
2. Gradient Boosting -                                                  
In gradient boosting, the decision tree works on all depths and you can create as many children you want.                     

Example Scenario -                                                   
Q   O/P  Weight(Amount of time spent studying each question)                                                                                              
1    ✔    1/5                                                            
2    ✔    1/5                                                              
3    ✔    1/5                                                           
4    X     1/5                                                                
5    X     1/5                                                          
Misclassification Rate - 2/5 = 0.4                                               
stage = log(1-0.4/0.4) ≈ log(0.6/0.4) ≈ 0.4                                                       
Wnew = Wold x e^stage                                       
Wnew = 1/5 x e^0.4  ≈ 0.29

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("bank.csv")
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,0,1,1,0,2343,1,0,2,5,8,1042,1,-1,0,3,1
1,56,0,1,1,0,45,0,0,2,5,8,1467,1,-1,0,3,1
2,41,9,1,1,0,1270,1,0,2,5,8,1389,1,-1,0,3,1
3,55,7,1,1,0,2476,1,0,2,5,8,579,1,-1,0,3,1
4,54,0,1,2,0,184,0,0,2,5,8,673,2,-1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,2,0,0,1,1,0,0,20,0,257,1,-1,0,3,0
11158,39,7,1,1,0,733,0,0,2,16,6,83,4,-1,0,3,0
11159,32,9,2,1,0,29,0,0,0,19,1,156,2,-1,0,3,0
11160,43,9,1,1,0,0,0,1,0,8,8,9,2,172,5,0,0


In [3]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'deposit'],
      dtype='object')

In [4]:
df.describe()
#poutcome and month are negatively skewed.
#pdays, duration, balance, age are positively skewed.

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,4.487905,1.199337,1.285164,0.015051,1528.538524,0.473123,0.130801,0.489697,15.658036,5.445709,371.993818,2.508421,51.330407,0.832557,2.477782,0.473840
std,11.913369,3.225132,0.625552,0.749478,0.121761,3225.413326,0.499299,0.337198,0.818724,8.420740,3.191939,347.128386,2.722077,108.758282,2.292007,1.002952,0.499338
min,18.000000,0.000000,0.000000,0.000000,0.000000,-6847.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
25%,32.000000,1.000000,1.000000,1.000000,0.000000,122.000000,0.000000,0.000000,0.000000,8.000000,3.000000,138.000000,1.000000,-1.000000,0.000000,2.000000,0.000000
50%,39.000000,4.000000,1.000000,1.000000,0.000000,550.000000,0.000000,0.000000,0.000000,15.000000,6.000000,255.000000,2.000000,-1.000000,0.000000,3.000000,0.000000
75%,49.000000,7.000000,2.000000,2.000000,0.000000,1708.000000,1.000000,0.000000,1.000000,22.000000,8.000000,496.000000,3.000000,20.750000,1.000000,3.000000,1.000000
max,95.000000,11.000000,2.000000,3.000000,1.000000,81204.000000,1.000000,1.000000,2.000000,31.000000,11.000000,3881.000000,63.000000,854.000000,58.000000,3.000000,1.000000


In [5]:
df.info()
#there are 17 non null integer columns.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        11162 non-null  int64
 1   job        11162 non-null  int64
 2   marital    11162 non-null  int64
 3   education  11162 non-null  int64
 4   default    11162 non-null  int64
 5   balance    11162 non-null  int64
 6   housing    11162 non-null  int64
 7   loan       11162 non-null  int64
 8   contact    11162 non-null  int64
 9   day        11162 non-null  int64
 10  month      11162 non-null  int64
 11  duration   11162 non-null  int64
 12  campaign   11162 non-null  int64
 13  pdays      11162 non-null  int64
 14  previous   11162 non-null  int64
 15  poutcome   11162 non-null  int64
 16  deposit    11162 non-null  int64
dtypes: int64(17)
memory usage: 1.4 MB


In [6]:
features=df.iloc[:,:-1]
target=df.iloc[:,-1]

In [8]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(features,target,test_size=0.3,random_state=1)

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

ada = AdaBoostClassifier()
ada.fit(xtrain,ytrain)
ypred = ada.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84      1760
           1       0.83      0.80      0.81      1589

    accuracy                           0.83      3349
   macro avg       0.83      0.83      0.83      3349
weighted avg       0.83      0.83      0.83      3349



In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

ada = AdaBoostClassifier(n_estimators=100)
ada.fit(xtrain,ytrain)
ypred = ada.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1760
           1       0.83      0.80      0.82      1589

    accuracy                           0.83      3349
   macro avg       0.83      0.83      0.83      3349
weighted avg       0.83      0.83      0.83      3349



In [19]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=180)
gbc.fit(xtrain,ytrain)
ypred = gbc.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1760
           1       0.82      0.87      0.84      1589

    accuracy                           0.85      3349
   macro avg       0.85      0.85      0.85      3349
weighted avg       0.85      0.85      0.85      3349



In [24]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 2.2 MB/s eta 0:00:46
   ---------------------------------------- 0.2/99.8 MB 2.4 MB/s eta 0:00:43
   ---------------------------------------- 0.2/99.8 MB 2.6 MB/s eta 0:00:38
   ---------------------------------------- 0.2/99.8 MB 2.6 MB/s eta 0:00:38
   ---------------------------------------- 0.4/99.8 MB 1.8 MB/s eta 0:00:55
   ---------------------------------------- 0.4/99.8 MB 1.7 MB/s eta 0:00:59
   ---------------------------------------- 0.5/99.8 MB 1.5 MB/s eta 0:01:06
   ---------------------------------------- 0.5/99.8 MB 1.4 MB/s eta 0:01:11
   ---------------------------------------- 0.5/99.8 MB 1.3 MB/s eta 0:01:16
   -----------------------------

   ------------- -------------------------- 32.6/99.8 MB 120.6 kB/s eta 0:09:17
   ------------- -------------------------- 32.6/99.8 MB 120.6 kB/s eta 0:09:17
   ------------- -------------------------- 32.6/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.6/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.6/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.8 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.7 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.7 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 120.7 kB/s eta 0:09:16
   ------------- -------------------------- 32.7/99.8 MB 121.2 kB/s eta 0:09:14
   ------------- -----------------------

In [25]:
from xgboost import XGBClassifier

In [26]:
xgb = XGBClassifier()
xgb.fit(xtrain,ytrain)
ypred = xgb.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.88      0.84      0.86      1760
           1       0.83      0.87      0.85      1589

    accuracy                           0.85      3349
   macro avg       0.85      0.85      0.85      3349
weighted avg       0.85      0.85      0.85      3349

